In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#prp_df is the dataframe containing the main csv file dataframe
prp_df = pd.read_csv("../input/delhi-house-price-prediction/MagicBricks.csv")
prp_df.head()

In [ ]:
# This contains 11 columns and 1259 rows 
print(prp_df.shape)
# This is used to find that dataframe contains some nan values
print(prp_df.isnull().values.any())

# Try to find which columns contains nan values
prp_df.columns

for i in prp_df.columns:
    print(i+' ' , end='')
    print(prp_df[i].isnull().values.any())
    
# Now will see how many rows contains nan values
# we get that 254 rows have some nan values
print()
nan_val=sum([True for idx,row in prp_df.iterrows() if any(row.isnull())])
print(nan_val)

# That is around 20 percent of dataset rows have nan values. 
print(nan_val/prp_df.shape[0])

In [ ]:
### We have issue in parking add mean in nan val or median =1
# bathroom add mean in nan val or median =2
# Furnishing and Type drop missing values
# Locality
# Per_Sqft might we can do is area/price for that column

#Handling missing values

prp_df['Bathroom'] = prp_df['Bathroom'].fillna(2)
prp_df['Parking'] = prp_df['Parking'].fillna(1)

prp_df = prp_df[prp_df['Furnishing'].notna()]
prp_df = prp_df[prp_df['Type'].notna()]

prp_df['Per_Sqft'].fillna(prp_df['Price']/prp_df['Area'], inplace=True)

print(prp_df.shape)
for i in prp_df.columns:
    print(i+' ' , end='')
    print(prp_df[i].isnull().values.any())

In [ ]:
# Missing values are successfully handled
prp_df.head()

In [ ]:
# to see the correlation between we need to remove or convert categorical variables
#instead of this we can directly replace cat with a val for each col

cleanup_nums = {"Status":     {"Ready_to_move": 0, "Almost_ready": 1},"Furnishing":{"Semi-Furnished": 0, "Furnished": 1,"Unfurnished":2}, 
                "Transaction":     {"New_Property": 0, "Resale": 1}, "Type":     {"Builder_Floor": 0, "Apartment": 1}}

prp_df = prp_df.replace(cleanup_nums)
prp_df.head()
    

In [ ]:
df=prp_df.corr()
df

In [ ]:
#let's create a heat map to see the correlation between different columns
import seaborn as sns
import matplotlib.pyplot as plt
f,ax=plt.subplots(figsize=(15, 10))
df=prp_df.corr()
sns.heatmap(df)

#Hence we can observe strong correlation between  Area, BHK,bathroom, persqft

In [ ]:
#pd.unique(prp_df['Locality'])
#prp_df.drop('Locality',axis='columns', inplace=True)
#based on correlation we can drop
prp_df.drop('Furnishing',axis='columns', inplace=True)
prp_df.drop('Parking',axis='columns', inplace=True)


In [ ]:
# #locality obs

#location_stats = prp_df.groupby('Locality')['Locality'].agg('count').sort_values(ascending = False)
#location_stats
# len(pd.unique(prp_df['Locality']))
# #pd.unique(prp_df['Locality'])

# lis=[]
# for i in prp_df['Locality']:
#     temp_i=i.split()
#     #sw=i.split(' ', 1)[1]
#     fw=temp_i[0]
#     #sw=temp_i[1]
#     i=fw
#     lis.append(i)

# #print(set(lis))


# prp_df['Locality']=lis

# prp_df.head()
# # #len(pd.unique(lis))

location_stats = prp_df.groupby('Locality')['Locality'].agg('count').sort_values(ascending = False)
location_stats

#reduced the unique values from 363 to 199

# Classify less than 20 as others
len(location_stats[location_stats < 11])

#reduced the unique values from 199 to 22
location_stas_less_than = location_stats[location_stats < 11]
prp_df.Locality = prp_df.Locality.apply(lambda x : 'Other' if x in location_stas_less_than else x)


prp_df.Locality = prp_df.Locality.apply(lambda x : 'J R Designers Floors' if x =='J' else x)

# prp_df.head()
# len(pd.unique(prp_df['Locality']))


In [ ]:
len(pd.unique(prp_df['Locality']))

In [ ]:
#Assign numerical value to locality
list_prp=[]
list_prp=list(prp_df.Locality.unique())
list_prp

prp_df['Locality'] = prp_df['Locality'].astype('category')
prp_df['Locality']=prp_df['Locality'].cat.codes
prp_df.Locality.unique()

In [ ]:
prp_df['BHK']=prp_df['BHK'].astype(float)
prp_df['Locality']=prp_df['Locality'].astype(float)


In [ ]:
X = prp_df.drop(['Price'],axis='columns')
print(X.head(3))

y = prp_df.Price
print(y.head(3))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)


In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)


0.4814762972281953 accuracy(without locality)
0.4817679934752995 accuracy (with locality 120 class)
0.4821458736518861 accuracy (with locality 360 class) 
0.7382633342552736 accuracy (after removing outlier from area and per_sqrt) 


In [ ]:
#Outlier detection and handling
import matplotlib

flag=0
for i in list(prp_df.Per_Sqft):
    if i>40000:
        flag=flag+1
print(flag)
prp_df = prp_df[prp_df['Per_Sqft'].values <40000]
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(prp_df.Per_Sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")
print(len(list(prp_df.Per_Sqft)))

#prp_df['Per_Sqft']= np.sqrt(prp_df['Per_Sqft'])
#Normalise each column
#prp_df['Per_Sqft']=(prp_df['Per_Sqft']-prp_df['Per_Sqft'].mean())/prp_df['Per_Sqft'].std()

In [ ]:
#Outlier detection and handling
import matplotlib
flag=0
for i in list(prp_df.Area):
    if i>6000:
        flag=flag+1
print(flag)
prp_df = prp_df[prp_df['Area'].values <6000]
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(prp_df.Area,rwidth=0.8)
plt.xlabel("Area")
plt.ylabel("Count")
print(len(list(prp_df.Area)))


#prp_df['Area']=np.sqrt(prp_df['Area'])
#Normalise each column
#prp_df['Area']=(prp_df['Area']-prp_df['Area'].mean())/prp_df['Area'].std()


In [ ]:
X = prp_df.drop(['Price'],axis='columns')
print(X.head(3))
y = prp_df.Price
print(y.head(3))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)





In [ ]:
from sklearn.linear_model import Ridge
lr_clf = Ridge(alpha=0.1)
lr_clf.fit(X_train,y_train)
print(lr_clf.score(X_test,y_test))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
degree=2
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(X_train,y_train)
print(polyreg.score(X_test,y_test))